# Automatic emittance measurement at LCLS-II
In this case we have 4 quadrupoles, 3 that are used to minimize the incoming beam and
 the fourth to perform the quad scan measurement.

In [9]:
# optionally add scripts location to path
if True:
    import sys
    sys.path.append("../../../")

import xopt
print(xopt.__version__)

# set up data saving locations
data_dir = "/home/physics3/ml_tuning/20230729_LCLS_Injector"

run_name = "optimize_1"
run_dir = f"{data_dir}/{run_name}"
import os
if not os.path.exists(run_dir):
    os.mkdir(run_dir)

In [ ]:
from scripts.evaluate_function.screen_image import measure_beamsize, measure_background
from scripts.optimize_function import optimize_function
from scripts.characterize_emittance import characterize_emittance

In [ ]:
from utils import VARIABLE_RANGES, SCAN_VARIABLE, \
    MEASUREMENT_OPTIONS, IMAGE_CONSTRAINTS, TUNING_VARIABLES, SCREEN_NAME, eval_beamsize

## (Optional) Measure background image

In [ ]:
MEASURE_BACKGROUND = True
if MEASURE_BACKGROUND:
    measure_background(SCREEN_NAME)
    BACKGROUND_FILE = f"{SCREEN_NAME}_background.npy".replace(":","_")
else:
    BACKGROUND_FILE = None

In [ ]:
# verify background image
import numpy as np
import matplotlib.pyplot as plt
plt.imshow(np.load(BACKGROUND_FILE))

## Optimize upstream beam parameters to minimze beamsize
Note that the scan variable is fixed at zero for maximum range quadrupole scan.

In [ ]:
from xopt import VOCS
opt_vocs = VOCS(
        variables = {ele: VARIABLE_RANGES[ele] for ele in TUNING_VARIABLES},
        constants = {SCAN_VARIABLE: 0.0} | MEASUREMENT_OPTIONS,
        constraints = IMAGE_CONSTRAINTS,
        objectives = {"total_size": "MINIMIZE"}
    )

opt_x = optimize_function(
    opt_vocs, eval_beamsize, n_iterations=1,
)

In [ ]:
opt_x.data

## Automatic quad scan and emittance characterization

In [ ]:
from emitopt.utils import get_quad_strength_conversion_factor
from utils import BEAM_ENERGY, QUAD_LENGTH, DRIFT_LENGTH, PV_TO_INTEGRATED_GRADIENT, eval_beamsize
emit_vocs = VOCS(
        variables = {SCAN_VARIABLE: VARIABLE_RANGES[SCAN_VARIABLE]},
        observables = ["S_x_mm", "S_y_mm"],
        constraints = IMAGE_CONSTRAINTS,
        constants = MEASUREMENT_OPTIONS
    )

# create conversion factor from PV value to geometric focusing strength
integrated_gradient_to_geometric_focusing_strength = get_quad_strength_conversion_factor(
        BEAM_ENERGY, QUAD_LENGTH
    )
quad_strength_scale_factor =  PV_TO_INTEGRATED_GRADIENT * integrated_gradient_to_geometric_focusing_strength

emit_results, emit_Xopt = characterize_emittance(
    emit_vocs,
    eval_beamsize,
    QUAD_LENGTH,
    DRIFT_LENGTH,
    BEAM_ENERGY,
    quad_strength_key=SCAN_VARIABLE,
    quad_strength_scale_factor=quad_strength_scale_factor,
    rms_x_key="S_x_mm",
    rms_y_key="S_y_mm",
    quad_scan_analysis_kwargs={"visualize": True}
)

In [ ]:
emit_results

In [ ]:
emit_Xopt.data